# WSI 2022L
# Laboratorium 3 (Dwuosobowe gry deterministyczne)
# Michał Brus, 299106

## Czysty algorytm minimax z obcięciem alfa/beta

In [8]:
def alpha_beta(position, depth, alpha, beta, max_player):
    if depth == 0 or position == 'terminal':
        return evaluate(position)
    if current_player is max_player:
        max_eval = -math.inf
        for child in children(position):
            eval = alpha_beta(child, depth - 1, alpha, beta, False)
            max_eval = max(max_eval, eval)
            alpha = max(alpha, eval)
            if beta <= alpha:
                break
        return max_eval
    else:
        min_eval = math.inf
        for child in children(position):
            eval = alpha_beta(child, depth - 1, alpha, beta, True)
            min_eval = min(min_eval, eval)
            beta = min(beta, eval)
            if beta <= alpha:
                break
        return min_eval


## Algorytm minimax z obcięciem alfa/beta (dostosowany do gry ConnectFour)

In [9]:
import math

import numpy as np


def evaluate(game_state):
    return game_state.get_heuristic()


def get_winner(game_state):
    return game_state.get_winner()


def current_player(game_state):
    return game_state.get_current_player()


def is_finished(game_state):
    return game_state.is_finished()


# def make_children(game_state):
#     possible_moves = game_state.get_moves()
#     possible_states = []
#     for move in possible_moves:
#         possible_states += [game_state.make_move(move)]
#     return possible_states, possible_moves


def alpha_beta(game_state, depth, alpha, beta, maximising_player):
    if depth == 0 or is_finished(game_state):
        if is_finished(game_state):
            winner = get_winner(game_state)
            if winner is current_player(game_state):
                return 10e6, None
            elif winner is not (current_player(game_state) or None):
                return -10e6, None
            else:
                return 0, None
        else:
            return evaluate(game_state), None
    if maximising_player:
        max_eval = -math.inf
        possible_moves = game_state.get_moves()
        rng = np.random.default_rng()
        rng.shuffle(possible_moves)
        best_move = rng.choice(possible_moves)
        for move in possible_moves:
            state = game_state.make_move(move)
            eval = alpha_beta(state, depth - 1, alpha, beta, False)[0]
            if eval > max_eval:
                max_eval = eval
                best_move = move
            elif max_eval == eval:
                rng = np.random.default_rng()
                best_move = rng.choice([best_move, move])
            alpha = max(alpha, max_eval)
            if alpha >= beta:
                break

        return max_eval, best_move
    else:
        min_eval = math.inf
        possible_moves = game_state.get_moves()
        rng = np.random.default_rng()
        rng.shuffle(possible_moves)
        best_move = rng.choice(possible_moves)
        for move in possible_moves:
            state = game_state.make_move(move)
            eval = alpha_beta(state, depth - 1, alpha, beta, True)[0]
            if eval < min_eval:
                min_eval = eval
                best_move = move
            elif min_eval == eval:
                rng = np.random.default_rng()
                best_move = rng.choice([best_move, move])
            min_eval = min(min_eval, eval)
            beta = min(beta, min_eval)
            if alpha >= beta:
                break
        return min_eval, best_move


### Do oceny stanu planszy służy funkcja heurysytczna dodana do kodu gry w pliku: games_examples/two_player_games/games/connect_four.py

Funkcja get_heuristic() korzysta z funkcji pomocniczej _heuristic()

## Uruchomienie gry

In [10]:
import c4_alfa_beta as ai
import math

from games_examples.two_player_games.player import Player
from games_examples.two_player_games.games.connect_four import ConnectFour, ConnectFourMove, ConnectFourState


def run(depth_1, depth_2, with_print, interactive):
    p1 = Player('a')
    p2 = Player('b')
    game = ConnectFour(first_player=p1, second_player=p2)

    if with_print:
        print(str(game))
    while not game.is_finished():
        game_state = game.state
        if interactive:
            a_points, a_move = ai.alpha_beta(game_state, depth_1, -math.inf, math.inf, True)
            game.make_move(a_move)
            print(f"Wartość ruchu a: {a_points} \n")
            print(str(game))
            b_move = ConnectFourMove(int(input("Podaj nr kolumny:")))
            game.make_move(b_move)
            print(str(game))

        else:
            a_points, a_move = ai.alpha_beta(game_state, depth_1, -math.inf, math.inf, True)
            b_points, b_move = ai.alpha_beta(game_state, depth_2, -math.inf, math.inf, False)
            if with_print:
                game.make_move(a_move)
                print(f"Wartość ruchu a: {a_points} \n")
                print(str(game))
                game.make_move(b_move)
                print(f"Wartość ruchu b: {b_points} \n")
                print(str(game))
            else:
                game.make_move(a_move)
                game.make_move(b_move)

    winner = game.get_winner()
    if winner == p1:
        if with_print:
            print("\nWygrywa a!")
        return 'p1'
    elif winner == p2:
        if with_print:
            print("\nWygrywa b!")
        return 'p2'
    else:
        if with_print:
            print("\nRemis!")
        return 'r'

Gra może zostać uruchomiona w trybie interaktywnym, gdzie gracz gra z AI lub dwa AI przeciwko sobie. Można też włączyć opcję wyświetlania wyników na konsoli (opcja print).

## Przykładowe uruchomienie z opcją print dla depth_1 = 1, depth_2 = 2

In [11]:
run(1, 2, True, False)

Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
Wartość ruchu a: 0 

Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][a][ ][ ]
Wartość ruchu b: 0 

Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][a][ ][b]
Wartość ruchu a: 0 

Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][a][ ][b]
Wartość ruchu b: 0 

Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][b]
[ ][ ][ ][a][a][ ][b]
Wartość ruchu a: 2 

Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][a][ ][b]
[ ][ ][ ][a][a][ 

'p2'

## Testy i statystyki

In [12]:
def make_statistics(depth_1, depth_2, to_print):
    runs_number = 30
    sum_a = sum_b = sum_r = 0
    for i in range(runs_number):
        winner = run(depth_1, depth_2, False, False)
        if winner == 'p1':
            sum_a += 1
        elif winner == 'p2':
            sum_b += 1
        else:
            sum_r += 1
    a_pct = round(100 * sum_a / runs_number)
    b_pct = round(100 * sum_b / runs_number)
    r_pct = round(100 * sum_r / runs_number)
    if to_print:
        print(
            f"Wygrane a: {a_pct}%, wygrane b: {b_pct}%, remisy: {r_pct}%.")
    return a_pct, b_pct, r_pct

Funkcja ta uruchamia 30 gier, aby sprawdzić, który gracz wygrywa procentowo w grach o podanych parametrach (depth_1, depth_2). Obydwaj gracze to AI korzystający z algorytmu alpha_beta.

In [ ]:
make_statistics(1, 1, False)

In [27]:
data = np.empty([5, 5])
for i in range(5):
    for j in range (5):
        print(f"Dla (d1, d2) = {i+1, j+1}:")
        result = make_statistics(i+1, j+1, True)
        data [i, j] = result[0]
    print("\n")

Dla (d1, d2) = (1, 1):
Wygrane a: 0%, wygrane b: 100%, remisy: 0%.
Dla (d1, d2) = (1, 2):
Wygrane a: 3%, wygrane b: 97%, remisy: 0%.
Dla (d1, d2) = (1, 3):
Wygrane a: 0%, wygrane b: 100%, remisy: 0%.
Dla (d1, d2) = (1, 4):
Wygrane a: 0%, wygrane b: 100%, remisy: 0%.
Dla (d1, d2) = (1, 5):
Wygrane a: 0%, wygrane b: 100%, remisy: 0%.


Dla (d1, d2) = (2, 1):
Wygrane a: 10%, wygrane b: 73%, remisy: 17%.
Dla (d1, d2) = (2, 2):
Wygrane a: 30%, wygrane b: 57%, remisy: 13%.
Dla (d1, d2) = (2, 3):
Wygrane a: 63%, wygrane b: 30%, remisy: 7%.
Dla (d1, d2) = (2, 4):
Wygrane a: 67%, wygrane b: 30%, remisy: 3%.
Dla (d1, d2) = (2, 5):
Wygrane a: 70%, wygrane b: 30%, remisy: 0%.


Dla (d1, d2) = (3, 1):
Wygrane a: 3%, wygrane b: 83%, remisy: 13%.
Dla (d1, d2) = (3, 2):
Wygrane a: 50%, wygrane b: 43%, remisy: 7%.
Dla (d1, d2) = (3, 3):
Wygrane a: 20%, wygrane b: 73%, remisy: 7%.
Dla (d1, d2) = (3, 4):
Wygrane a: 40%, wygrane b: 60%, remisy: 0%.
Dla (d1, d2) = (3, 5):
Wygrane a: 30%, wygrane b: 60%, re

W przypadku, gdy gracz A grał na głębkości 1, gracz B wygrywał zawsze. Gdy A grał na głębokości 2, wtedy B zwiększając głębokość zmniejszał szanse na wygraną. Dla głębokości 3, 4, 5 wystąpiły duże rozbieżności i nie widać jednoznacznego trendu.